# Proyecto 1

**Juan David Guevara Arévalo - 202116875**

## Instalación de librerías

In [2]:
%pip install pandas numpy matplotlib seaborn nltk scikit-learn scikit-plot scipy ydata-profiling unidecode

  Using cached matplotlib-3.9.4-cp39-cp39-win_amd64.whl (7.8 MB)
  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
  Using cached scikit_plot-0.3.7-py3-none-any.whl (33 kB)
  Using cached ydata_profiling-4.12.2-py2.py3-none-any.whl (390 kB)
  Using cached wordcloud-1.9.4-cp39-cp39-win_amd64.whl (300 kB)
  Using cached phik-0.12.4-cp39-cp39-win_amd64.whl (666 kB)
  Using cached ImageHash-4.3.1-py2.py3-none-any.whl (296 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\estiv\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
import sys

import re, string
import unidecode
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.base import BaseEstimator, ClassifierMixin

import matplotlib.pyplot as plt

In [4]:
import nltk

nltk.download("punkt_tab")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\estiv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\estiv\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\estiv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## 1. Entendimiento y preparación de datos

In [6]:
df = pd.read_csv("data/fake_news_spanish.csv", sep=";", encoding="utf-8")

In [7]:
df

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018
...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022


In [8]:
df["Label"].value_counts()

Label
1    33158
0    23905
Name: count, dtype: int64

In [9]:
from scipy import stats as st

texts = df.copy()
texts["count"] = [len(x) for x in texts["Descripcion"]]
texts["max"] = [max(len(x) for x in i.split(" ")) for i in texts["Descripcion"]]
texts["min"] = [min(len(x) for x in i.split(" ")) for i in texts["Descripcion"]]
texts["palabra_max"] = [max(i.split(" "), key=len) for i in texts["Descripcion"]]
texts["palabra_min"] = [min(i.split(" "), key=len) for i in texts["Descripcion"]]
texts

,ID,Label,Titulo,Descripcion,Fecha,count,max,min,palabra_max,palabra_min
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,218,13,1,ultraderecha.,y
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,3251,17,1,encarcelarlo.\nEPN,A
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,169,13,2,valencianismo,El
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,133,15,2,"parlamentarios,",En
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,242,14,1,desdramatizado,a
...,...,...,...,...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,244,12,1,"contemplada,",y
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,314,11,1,democrática,a
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,70,11,2,coordinador,El
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,232,12,1,corresponda.,y


In [11]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        if word is not None:
          new_word = unidecode.unidecode(word)
          new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    return [word.lower() for word in words]

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        if word is not None:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
    return new_words


def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    sw = set(stopwords.words('spanish'))
    for word in words:
        if word not in sw:
            new_words.append(word)
    return new_words


def preprocessing(words):
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words

In [22]:
texts["words"] = texts["Descripcion"].apply(word_tokenize)
texts["words"] = texts["words"].apply(preprocessing)

In [23]:
texts

,ID,Label,Titulo,Descripcion,Fecha,count,max,min,palabra_max,palabra_min,words,joined
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,218,13,1,ultraderecha.,y,"[diario, britanico, publico, pasado, jueves, e...",diario britanico publico pasado jueves editori...
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,3251,17,1,encarcelarlo.\nEPN,A,"[revelan, gobierno, negocio, liberacion, mirel...",revelan gobierno negocio liberacion mireles ca...
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,169,13,2,valencianismo,El,"[valencianismo, convoca, castello, fiesta, gra...",valencianismo convoca castello fiesta grande c...
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,133,15,2,"parlamentarios,",En,"[politica, igual, negociar, empresarios, negoc...",politica igual negociar empresarios negociar g...
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,242,14,1,desdramatizado,a,"[entrevista, punt, avui, lider, jxcat, desdram...",entrevista punt avui lider jxcat desdramatizad...
...,...,...,...,...,...,...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,244,12,1,"contemplada,",y,"[gobierno, regional, indicado, atencion, dia, ...",gobierno regional indicado atencion dia incluy...
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,314,11,1,democrática,a,"[si, higiene, democratica, lleva, exigencia, t...",si higiene democratica lleva exigencia tener p...
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,70,11,2,coordinador,El,"[coordinador, federal, iu, asegura, monarquia,...",coordinador federal iu asegura monarquia putre...
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,232,12,1,corresponda.,y,"[santiago, abascal, vox, derecho, deber, forma...",santiago abascal vox derecho deber formar gobi...


In [ ]:
%pip install spacy



     --------------------------------------- 12.3/12.3 MB 34.5 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 46.7 MB/s eta 0:00:00
     ---------------------------------------- 44.9/44.9 KB ? eta 0:00:00
     ---------------------------------------- 122.7/122.7 KB ? eta 0:00:00
     ---------------------------------------- 50.3/50.3 KB 2.5 MB/s eta 0:00:00
     ------------------------------------- 183.0/183.0 KB 10.8 MB/s eta 0:00:00
     ------------------------------------- 633.4/633.4 KB 41.5 MB/s eta 0:00:00
     ---------------------------------------- 5.4/5.4 MB 34.5 MB/s eta 0:00:00
     ---------------------------------------- 6.2/6.2 MB 26.7 MB/s eta 0:00:00
     ---------------------------------------- 242.4/242.4 KB ? eta 0:00:00
     ---------------------------------------- 61.7/61.7 KB ? eta 0:00:00
     ---------------------------------------- 52.5/52.5 KB ? eta 0:00:00
     -------------------------------------- 152.1/152.1 KB 9.5 MB/s eta 0

You should consider upgrading via the 'C:\Users\estiv\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [31]:
!python -m spacy download es_core_news_sm

     --------------------------------------- 12.9/12.9 MB 36.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


You should consider upgrading via the 'C:\Users\estiv\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [32]:
import spacy
nlp = spacy.load("es_core_news_sm")


def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    doc = nlp(" ".join(words))  # Convertir la lista en texto para procesarla con spaCy
    lemmatized_words = [token.lemma_ if token.pos_ == "VERB" else token.text for token in doc]
    return lemmatized_words

In [33]:
texts["words"] = texts["words"].apply(lemmatize_verbs)
texts

,ID,Label,Titulo,Descripcion,Fecha,count,max,min,palabra_max,palabra_min,words,joined
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,218,13,1,ultraderecha.,y,"[diario, britanico, publico, pasado, jueves, e...",diario britanico publico pasado jueves editori...
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,3251,17,1,encarcelarlo.\nEPN,A,"[revelar, gobierno, negocio, liberacion, mirel...",revelan gobierno negocio liberacion mireles ca...
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,169,13,2,valencianismo,El,"[valencianismo, convocar, castello, fiesta, gr...",valencianismo convoca castello fiesta grande c...
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,133,15,2,"parlamentarios,",En,"[politica, igual, negociar, empresarios, negoc...",politica igual negociar empresarios negociar g...
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,242,14,1,desdramatizado,a,"[entrevista, punt, avui, lider, jxcat, desdram...",entrevista punt avui lider jxcat desdramatizad...
...,...,...,...,...,...,...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,244,12,1,"contemplada,",y,"[gobierno, regional, indicado, atencion, dia, ...",gobierno regional indicado atencion dia incluy...
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,314,11,1,democrática,a,"[si, higienir, democratica, llevar, exigencia,...",si higiene democratica lleva exigencia tener p...
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,70,11,2,coordinador,El,"[coordinador, federal, iu, asegurar, monarquia...",coordinador federal iu asegura monarquia putre...
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,232,12,1,corresponda.,y,"[santiago, abascal, vox, derecho, deber, forma...",santiago abascal vox derecho deber formar gobi...


In [14]:
all_words = nltk.FreqDist(word for tokens in texts['words'] for word in tokens)
print(all_words.most_common(100))

[('gobierno', 11361), ('mas', 7006), ('presidente', 6262), ('partido', 6217), ('pp', 5758), ('madrid', 4850), ('catalunya', 4812), ('si', 4474), ('iniciativa', 3774), ('tras', 3577), ('per', 3429), ('dos', 3416), ('ejecutivo', 3349), ('lider', 3342), ('congreso', 3257), ('tambien', 3254), ('equo', 3218), ('vers', 3103), ('psoe', 3079), ('ley', 3062), ('formacion', 3037), ('anos', 3019), ('podemos', 2812), ('elecciones', 2600), ('espana', 2587), ('ser', 2476), ('general', 2416), ('acuerdo', 2405), ('publico', 2398), ('portavoz', 2365), ('coalicion', 2335), ('unidas', 2295), ('despues', 2294), ('comunidad', 2280), ('politica', 2224), ('ciudadanos', 2178), ('pais', 2162), ('jose', 2161), ('asegura', 2128), ('nueva', 2104), ('pablo', 2084), ('tribunal', 2045), ('sanchez', 2039), ('parte', 1980), ('solo', 1976), ('tres', 1969), ('nacional', 1968), ('sido', 1917), ('ministra', 1909), ('segun', 1858), ('mientras', 1851), ('partidos', 1841), ('socialistas', 1805), ('caso', 1795), ('vox', 1773)

In [15]:
features = list(all_words)[:500]

In [16]:
def document_features(document, word_features=features):
    document_words = set(document)
    features = {}
    for word in word_features:
        features[f'contains({word})'] = (word in document_words)
    return features

In [17]:
data = [(document_features(tokens), label) for tokens, label in zip(texts['words'], texts['Label'])]

In [18]:
train_size = int(len(data) * 0.8)

In [19]:
from nltk import NaiveBayesClassifier, classify

train_set, test_set = data[:train_size], data[train_size:]

# 5. Train a Naive Bayes Classifier
classifier = NaiveBayesClassifier.train(train_set)

# 6. Evaluate the classifier
accuracy = classify.accuracy(classifier, test_set)
print(f'Accuracy: {accuracy:.2f}')

# 7. Show the most informative features
classifier.show_most_informative_features(10)

Accuracy: 0.74
Most Informative Features
        contains(eajpnv) = True                0 : 1      =    973.0 : 1.0
      contains(boluarte) = True                0 : 1      =    387.4 : 1.0
           contains(erc) = True                1 : 0      =    145.7 : 1.0
        contains(casado) = True                1 : 0      =    118.4 : 1.0
          contains(equo) = True                0 : 1      =     67.9 : 1.0
       contains(narbona) = True                0 : 1      =     58.1 : 1.0
           contains(per) = True                0 : 1      =     54.1 : 1.0
         contains(bildu) = True                1 : 0      =     53.5 : 1.0
       contains(podemos) = True                1 : 0      =     21.7 : 1.0
       contains(canaria) = True                0 : 1      =     20.9 : 1.0


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

vectorizer = TfidfVectorizer()
texts["joined"] = texts["words"].apply(lambda x: " ".join(x))
X = vectorizer.fit_transform(texts['joined'])
y = texts['Label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

clf = LogisticRegression(max_iter=10000)
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.7671592966878906


In [21]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf_svm = SVC(kernel='linear')  # Modelo SVM con kernel lineal
clf_svm.fit(X_train, y_train)   # Entrenar con los datos

predictions_svm = clf_svm.predict(X_test)  # Predecir etiquetas
accuracy_svm = accuracy_score(y_test, predictions_svm)  # Calcular precisión

print("SVM Accuracy:", accuracy_svm)


SVM Accuracy: 0.7736433202874


In [62]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(n_estimators=100, random_state=42)
clf_rf.fit(X_train, y_train)

predictions_rf = clf_rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, predictions_rf)
print("Random Forest Accuracy:", accuracy_rf)


Random Forest Accuracy: 0.704480401892634


In [68]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Crear y entrenar el modelo XGBoost
clf_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
clf_xgb.fit(X_train, y_train)

# Hacer predicciones
predictions_xgb = clf_xgb.predict(X_test)

# Calcular precisión
accuracy_xgb = accuracy_score(y_test, predictions_xgb)
print("XGBoost Accuracy:", accuracy_xgb)


ValueError: Please reshape the input data into 2-dimensional matrix.

## ZONA DE TESTING

In [ ]:
%pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 34.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [ ]:
import stanza

nlp = stanza.Pipeline(lang='es', processors='tokenize,lemma', use_gpu=True)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| lemma     | combined_nocharlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


In [ ]:
def preprocess_text(text):
    # Process the text through the pipeline
    doc = nlp(text)
    # Extract lemmas from each word in each sentence
    lemmas = [word.lemma for sentence in doc.sentences for word in sentence.words]
    # Join the lemmas back into a single string
    return ' '.join(lemmas)

texts['processed_text'] = texts['words'].apply(lambda x: " ".join(x))
processed_text = texts['Descripcion'].tolist()
#texts['processed_text'] = texts['Descripcion'].apply(preprocess_text)

In [ ]:
nlp(processed_text[0])

[
  [
    {
      "id": 1,
      "text": "El",
      "lemma": "el",
      "start_char": 0,
      "end_char": 2
    },
    {
      "id": 2,
      "text": "diario",
      "lemma": "diario",
      "start_char": 3,
      "end_char": 9
    },
    {
      "id": 3,
      "text": "británico",
      "lemma": "británico",
      "start_char": 10,
      "end_char": 19
    },
    {
      "id": 4,
      "text": "publicó",
      "lemma": "publicar",
      "start_char": 20,
      "end_char": 27
    },
    {
      "id": 5,
      "text": "este",
      "lemma": "este",
      "start_char": 28,
      "end_char": 32
    },
    {
      "id": 6,
      "text": "pasado",
      "lemma": "pasado",
      "start_char": 33,
      "end_char": 39
    },
    {
      "id": 7,
      "text": "jueves",
      "lemma": "jueves",
      "start_char": 40,
      "end_char": 46
    },
    {
      "id": 8,
      "text": "un",
      "lemma": "uno",
      "start_char": 47,
      "end_char": 49
    },
    {
      "id": 9,
      "text

In [ ]:
r = nlp.bulk_process(processed_text)

In [ ]:
stanza_processed = [[word.lemma.lower() for sentence in doc.sentences for word in sentence.words] for doc in r]

In [ ]:
texts["stanza_processed"] = stanza_processed

In [ ]:
vectorizer = TfidfVectorizer()
texts["joined_2"] = texts["stanza_processed"].apply(lambda x: " ".join(x))
X = vectorizer.fit_transform(texts['joined_2'])
y = texts['Label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.7724166131199253


In [ ]:
texts

,ID,Label,Titulo,Descripcion,Fecha,count,max,min,words,joined,processed_text,stanza_processed,joined_2
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,218,13,1,"[diario, britanico, publico, pasado, jueves, e...",diario britanico publico pasado jueves editori...,diario britanico publico pasado jueves editori...,"[el, diario, británico, publicar, este, pasado...",el diario británico publicar este pasado jueve...
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,3251,17,1,"[revelan, gobierno, negocio, liberacion, mirel...",revelan gobierno negocio liberacion mireles ca...,revelan gobierno negocio liberacion mireles ca...,"[revelan, que, el, gobierno, negocio, el, libe...",revelan que el gobierno negocio el liberación ...
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,169,13,2,"[valencianismo, convoca, castello, fiesta, gra...",valencianismo convoca castello fiesta grande c...,valencianismo convoca castello fiesta grande c...,"[el, valencianismo, convocar, en, castelló, su...",el valencianismo convocar en castelló su fiest...
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,133,15,2,"[politica, igual, negociar, empresarios, negoc...",politica igual negociar empresarios negociar g...,politica igual negociar empresarios negociar g...,"[en, política, ,, igual, que, haber, que, nego...","en política , igual que haber que negociar con..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,242,14,1,"[entrevista, punt, avui, lider, jxcat, desdram...",entrevista punt avui lider jxcat desdramatizad...,entrevista punt avui lider jxcat desdramatizad...,"[en, uno, entrevista, en, el, punt, avui, ,, e...","en uno entrevista en el punt avui , el líder d..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,244,12,1,"[gobierno, regional, indicado, atencion, dia, ...",gobierno regional indicado atencion dia incluy...,gobierno regional indicado atencion dia incluy...,"[el, gobierno, regional, haber, indicar, que, ...",el gobierno regional haber indicar que el aten...
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,314,11,1,"[si, higiene, democratica, lleva, exigencia, t...",si higiene democratica lleva exigencia tener p...,si higiene democratica lleva exigencia tener p...,"[si, el, higiene, democrático, yo, llevar, a, ...",si el higiene democrático yo llevar a ese exig...
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,70,11,2,"[coordinador, federal, iu, asegura, monarquia,...",coordinador federal iu asegura monarquia putre...,coordinador federal iu asegura monarquia putre...,"[el, coordinador, federal, de, iu, asegurar, q...",el coordinador federal de iu asegurar que el m...
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,232,12,1,"[santiago, abascal, vox, derecho, deber, forma...",santiago abascal vox derecho deber formar gobi...,santiago abascal vox derecho deber formar gobi...,"[santiago, abascal, :, vox, tener, el, derecho...",santiago abascal : vox tener el derecho y el d...


In [ ]:
all_words_2 = nltk.FreqDist(word for tokens in texts['stanza_processed'] for word in tokens)

data_2 = [(document_features(tokens), label) for tokens, label in zip(texts['stanza_processed'], texts['Label'])]

In [ ]:
features = list(all_words_2)[:500]

In [ ]:
train_set, test_set = data_2[:train_size], data_2[train_size:]

# 5. Train a Naive Bayes Classifier
classifier = NaiveBayesClassifier.train(train_set)

# 6. Evaluate the classifier
accuracy = classify.accuracy(classifier, test_set)
print(f'Accuracy: {accuracy:.2f}')

# 7. Show the most informative features
classifier.show_most_informative_features(10)

Accuracy: 0.71
Most Informative Features
      contains(boluarte) = True                0 : 1      =    366.0 : 1.0
           contains(erc) = True                1 : 0      =    145.3 : 1.0
        contains(casado) = True                1 : 0      =    118.6 : 1.0
          contains(equo) = True                0 : 1      =     67.8 : 1.0
       contains(narbona) = True                0 : 1      =     58.1 : 1.0
           contains(per) = True                0 : 1      =     54.1 : 1.0
         contains(bildu) = True                1 : 0      =     53.3 : 1.0
       contains(canaria) = True                0 : 1      =     23.6 : 1.0
         contains(david) = True                0 : 1      =     18.3 : 1.0
           contains(bng) = True                0 : 1      =     17.9 : 1.0
